<a href="https://colab.research.google.com/github/CS7140/PA-8/blob/main/Q1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Rajesh Sakhamuru

11-14-2020
# Matrix Factorization Recommender System

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# **From Data README:** 
## The dataset

The dataset files are modeled after the [MovieLens dataset] (http://www.grouplens.org/node/73) to make them as interchangeable as possible. There are three files: **users.dat**, **items.dat** and **ratings.dat**.

### ratings.dat

In this file the extracted ratings are stored in the following format: *user_id::movie_id::rating::rating_timestamp*. For example:

14927::0110912::9::1375657563

The ratings contained in the tweets are scaled from 0 to 10, as is the norm on the IMDb platform. To prevent information loss we have chosen to not down-scale this rating value, so all rating values of this dataset are contained in the interval [0,10].



In [ ]:
moviesData = pd.read_csv("drive/My Drive/Colab Notebooks/data/MovieRatings/movies.txt",  header=None, dtype=str, delimiter='::')
ratingsData = pd.read_csv("drive/My Drive/Colab Notebooks/data/MovieRatings/ratings.txt",  header=None, dtype=str, delimiter='::')
usersData = pd.read_csv("drive/My Drive/Colab Notebooks/data/MovieRatings/users.txt",  header=None, dtype=str, delimiter='::')

Only use the first 1000 ratings for the sake of faster calculations:

In [4]:
ratingsData = ratingsData[:1000]
ratingsData

,0,1,2,3
0,1,0114508,8,1381006850
1,2,0075314,1,1595468524
2,2,0102926,9,1590148016
3,2,0114369,10,1597555347
4,2,0118715,8,1596006798
...,...,...,...,...
995,74,6850820,5,1546759596
996,74,7668870,10,1546759474
997,74,7689964,8,1546759549
998,75,0816711,8,1387196996


### Represent the sparse User-MovieRatings matrix as a dictionary to save space:

In [5]:
userMovieRatings = {}

for idx, row in ratingsData.iterrows():
    if row[0] in userMovieRatings:
        userMovieRatings[row[0]][row[1]] = row[2]
    else:
        userMovieRatings[row[0]] = {row[1]:row[2]}


In [6]:
print("Movie Categories:", moviesData[2].str.split("|",expand=True,)[0].unique())

Movie Categories: ['Documentary' nan 'Short' 'Action' 'Drama' 'Adventure' 'Crime' 'Comedy'
 'Family' 'Biography' 'Fantasy' 'Animation' 'History' 'Horror' 'Romance'
 'Musical' 'Mystery' 'Western' 'Thriller' 'Sci-Fi' 'Film-Noir' 'War'
 'Adult' 'Music' 'Sport' 'News' 'Game-Show' 'Reality-TV' 'Talk-Show']


In [7]:
numUsers = len(ratingsData[0].unique())
numMovies = len(ratingsData[1].unique())

print("Number of Movies:", numMovies)
print("Number of Users: ", numUsers)

Number of Movies: 784
Number of Users:  75


Can change the number of latent factors calculated here:

In [8]:
latentFactors = 10

a = np.random.uniform(low=0, high=1, size=(numUsers, latentFactors))
usersFeatures = pd.DataFrame(a, index= range(1,numUsers+1))

b = np.random.uniform(low=0, high=1, size=(latentFactors, numMovies))
featuresMovies = pd.DataFrame(b, columns = ratingsData[1].unique())

### Matrix Factorization via Gradient Descent is below:
The implementation below, while functional, is incredibly ineffecient compared to the MXNet implementation in the textbook due to each cell's 2 gradients (one for each factor matrix) being calculated individually. If I ran this implementation with all 888452 movie ratings, the matrix would not be as sparse and the output matrices would be singificantly better, however it would take 3 days to run that given the inefficient solution. I ran this code using only 1000 ratings, which makes the ratings incredibly sparse, but as can be seen below, the UserXratings matrix (shown below) from the dot product of the factors does have the correct ratings for every movie that has been rated, although there are some concerning values such as ratings of slightly greater than 10 or less than 0 for movies which have not yet been rated by certain users.

Varying the size of the latent factors affects the accuracy of the predicted ratings. The number of latent factors corresponds to the number of features being mapped of the items (movies in this case) and users in the user-ratings matrix. If the number of latent factors is too low, then the factor matrices cannot adequately learn the user-movie ratings data, because the gradient descent cannot find a local minimum of error to map all of the movie ratings correctly. 

Additionally, if the number of latent factors is too high, then the categories/features being created/mapped could become too specific and after a certain threshold, which would be figured out via a grid search, there are diminishing returns for the predictions of ratings for movies that users have not yet watched. Also, an increase in latent factors can dramatically increase the epoch time for gradient descent causing the model to take much longer to converge to the factor matrices.

For this dataset I tried 5, 10, and 30 as different values for the number of latent factors, and 5 had too much error, and 30 did not provide any significant difference in performance compared to 10, except that it took far longer to calculate. 10 was quick for gradient descent and gave good results for the user-ratings matrix.



In [9]:
def predictRating(user:str, movie:str):
    return usersFeatures.loc[int(user)].dot(featuresMovies[movie])

def actualRating(userMovieRatings:dict, user:str, movie:str):
    if user in userMovieRatings and movie in userMovieRatings[user]:
        return int(userMovieRatings[user][movie])
    return None

# MATRIX FACTORIZATION WITH GRADIENT DESCENT
learningRate = 0.7
epochs = 250
for e in range(epochs):

    # updates every user-feature parameter based on the gradient
    for user in range(1, numUsers+1):
        for feature in range(latentFactors):
            userFeatureGradientSum = 0
            for movie in ratingsData[1].unique():
                actual = actualRating(userMovieRatings,str(user),movie)
                if actual is None:
                    continue
                predicted = predictRating(user,movie)
                userFeatureGradientSum += 2*(actual-predicted)*featuresMovies.at[feature,movie]
                
            userFeatureGradient = userFeatureGradientSum / numMovies
            usersFeatures.at[user,feature] += learningRate * userFeatureGradient
    
    # updates every feature-movie parameter based on the gradient
    for feature in range(latentFactors):
        for movie in ratingsData[1].unique():
            movieFeatureGradientSum = 0
            for user in range(1, numUsers+1):
                actual = actualRating(userMovieRatings,str(user),movie)
                if actual is None:
                    continue
                predicted = predictRating(user,movie)
                movieFeatureGradientSum += 2*(actual-predicted)*usersFeatures.at[user,feature]
                
            movieFeatureGradient = movieFeatureGradientSum / numUsers
            featuresMovies.at[feature,movie] += learningRate * movieFeatureGradient
    
    if e%40 == 0 and e != 0 and learningRate >= 0.1:
        learningRate /= 1.5

    if e%25 == 0:
        print("epoch #",e)        
        print("learningRate:", learningRate)
        print("Users x Features:")
        print(usersFeatures)
        print("Features x Movies:")
        print(featuresMovies)
        print("-----------------------------------------------")
print("-----------------------------------------------")
print("Users x Features:")
print(usersFeatures)
print("Features x Movies:")
print(featuresMovies)

epoch # 0
learningRate: 0.7
Users x Features:
           0         1         2  ...         7         8         9
1   0.042657  0.958150  0.062239  ...  0.762685  0.711409  0.413218
2   0.503466  1.033483  0.609218  ...  0.761107  0.316340  0.923152
3   0.839830  0.653078  0.872762  ...  0.585535  0.377411  0.100339
4   0.296332  0.766853  0.975789  ...  0.230653  0.371768  0.984456
5   0.753566  0.872466  0.094386  ...  0.956303  0.616232  0.807078
..       ...       ...       ...  ...       ...       ...       ...
71  0.478012  1.004800  0.127605  ...  0.442249  0.536534  0.932648
72  0.825761  0.174253  0.300343  ...  0.336923  0.519353  0.040143
73  0.749561  0.009003  0.710844  ...  0.279070  0.007884  0.860690
74  0.409638  0.033194  0.459787  ...  0.534943  0.296155  0.845233
75  0.792438  0.623777  0.931166  ...  0.725753  0.631572  0.174786

[75 rows x 10 columns]
Features x Movies:
    0114508   0075314   0102926  ...   7668870   7689964   1054606
0  0.507381  0.753934  0.814

In [14]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 10000)
pd.set_option('display.max_colwidth', 1000)
# Predicted Matrix:
print("Predicted Matrix:")
print(usersFeatures.dot(featuresMovies))
print()
# Actual Ratings:
print('Actual Ratings')
print(userMovieRatings)

Predicted Matrix:
      0114508   0075314    0102926    0114369   0118715   0120737   0208092   0358273   0477348  10039344   1051906   1568346    2278388   6199572   6723592   6751668   7131622   7975244   7984734   8367814    8579674   8946378   0790636    1800241    2278871    2395417   3344922   0267626    1343092   1477855   1920849   2024432   2084970   2884206    3040964   5022702    2378281    2980516   1001508   1142988   1292566   1355644   1441953   1454029   1878870   2250912   2543164   3783958   0237572   0810819    1392190    1663202   1895587   2080374   2381111   2494362   3076658   3532216    3659388   4934950   5052448    5657856   0317740   3612616    0361748   0481141   1282140   2388771    4477536   5213744    6450804   6957966    7286456   8201170   3226786    0181875    1024855   1025100    1386697    2277860   4196776    1790885    0110076    1188982   2574698    1843866   1872181   2103281    2235779    3322940    0437086    2380307    4520988   8079248    036